In [ ]:
# This cell does not get executed when run with Times Square
import os
import datetime

telescope = os.environ.get("SCHEDVIEW_TELESCOPE", "simonyi")
day_obs = int(os.environ.get("SCHEDVIEW_DAY_OBS", datetime.date.today().strftime("%Y%m%d")))
sim_date = datetime.date.fromisoformat(os.environ.get("SCHEDVIEW_SIM_DATE", datetime.date(day_obs//10000, (day_obs%10000)//100, day_obs%100).isoformat()))

"""
# To render this notebook to html, in the current directory with the current python environment

export SCHEDVIEW_SIM_DATE="2025-09-05"
export SCHEDVIEW_DAY_OBS="20250905"
export SCHEDVIEW_TELESCOPE="simonyi"
jupyter nbconvert \
    --to html \
    --execute \
    --no-input \
    --ExecutePreprocessor.kernel_name=python3 \
    --ExecutePreprocessor.startup_timeout=3600 \
    --ExecutePreprocessor.timeout=3600 \
    multiprenight.ipynb

# The public USDF page served at
#    https://usdf-rsp-int.slac.stanford.edu/schedview-static-pages/multiprenight/lsstcam/2025/08/18/multiprenight_2025-08-18.html
# is found at
#    /sdf/data/rubin/shared/scheduler/reports/multiprenight/lsstcam/2025/08/18/multiprenight_2025-08-18.html
"""
pass

In [ ]:
import sys
import os

#sched_source = 'env'
#sched_source = 'shared'
sched_source = 'devel'
match sched_source:
    case 'shared':
        if os.path.exists('/sdf/data/rubin/shared/scheduler/packages'):
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/uranography-1.2.1")
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/rubin_scheduler-3.12.1.dev16+g74bb601")
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/rubin_sim-2.2.4")
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/schedview-0.18.1.dev27+g2dced97")
    case 'devel':
        if os.path.exists('/sdf/data/rubin/user/neilsen/devel'):
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/uranography")
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/rubin_scheduler")
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/rubin_sim")
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/schedview")
    case _:
        # Use the current environment
        pass

In [ ]:
from schedview.util import config_logging_for_reports
import logging
config_logging_for_reports(logging.ERROR)

In [ ]:
from IPython.display import display, HTML, Markdown
import datetime
import math
import yaml
from urllib.parse import urlparse
import warnings
import itertools
import pandas as pd
import numpy as np
import astropy
import bokeh
import bokeh.io
import boto3
import colorcet
from erfa import ErfaWarning
from astropy.time import Time

In [ ]:
display(Markdown(f"# Pre-night briefing report for dayobs {day_obs} compaing different simulations for the {telescope} telescope, simulated on {sim_date}"))

In [ ]:
from sklearn.neighbors import KernelDensity

In [ ]:
usdf_sim_data_dir = "/sdf/data/rubin/shared/rubin_sim_data"
if os.path.exists(usdf_sim_data_dir):
    os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/data/rubin/shared/rubin_sim_data"

In [ ]:
import rubin_scheduler
import rubin_scheduler.utils
import rubin_sim.sim_archive
from rubin_scheduler.scheduler.model_observatory import ModelObservatory
from rubin_sim import maf
from lsst.resources import ResourcePath
from rubin_sim.sim_archive.vseqmetadata import VisitSequenceArchiveMetadata

In [ ]:
import schedview.compute
import schedview.compute.visits
import schedview.collect
import schedview.collect.rewards
import schedview.plot
import schedview.plot.rewards
from schedview import DayObs

In [ ]:
# Degraded IERS accuracy is never going to be important for these figures.

# If IERS degraded accuracy encountered, don't fail, just keep going.
astropy.utils.iers.conf.iers_degraded_accuracy = "ignore"

In [ ]:
bokeh.io.output_notebook()

In [ ]:
%matplotlib inline

In [ ]:
archive_uri = "s3://rubin:rubin-scheduler-prenight/opsim/"

if urlparse(archive_uri).scheme.upper() == 'S3':
    os.environ["LSST_DISABLE_BUCKET_VALIDATION"] = "1"
    os.environ["S3_ENDPOINT_URL"] = "https://s3dfrgw.slac.stanford.edu/"

In [ ]:
metadata_dsn = {
    'host': 'usdf-maf-visit-seq-archive-tx-ro.sdf.slac.stanford.edu',
    'user': 'reader',
    'database': 'opsim_log',
}
metadata_schema = 'vsmd'
visit_seq_archive_metadata = VisitSequenceArchiveMetadata(metadata_dsn, metadata_schema)

In [ ]:
day_obs_mjd = DayObs.from_date(day_obs).mjd
observatory = ModelObservatory(init_load_length=1)
timezone = "Chile/Continental"

## Astronomical events during the night

In [ ]:
day_obs_datetime = Time(day_obs_mjd, format='mjd').datetime
day_obs_date = datetime.date(day_obs_datetime.year, day_obs_datetime.month, day_obs_datetime.day)
night_events = schedview.compute.astro.night_events(day_obs_date)
night_events

## Sun and moon positions in the middle of the night

In [ ]:
model_observatory = ModelObservatory(init_load_length=1)
model_observatory.mjd = night_events.loc['night_middle', 'MJD']

In [ ]:
body_positions_wide = pd.DataFrame(model_observatory.almanac.get_sun_moon_positions(night_events.loc['night_middle', 'MJD']))
body_positions_wide.index.name = 'r'
body_positions_wide.reset_index(inplace=True)

angle_columns = ['RA', 'dec', 'alt', 'az']
all_columns = angle_columns + ['phase']
body_positions = (
    pd.wide_to_long(body_positions_wide, stubnames=('sun', 'moon'), suffix=r'.*', sep='_', i='r', j='')
    .droplevel('r')
    .T[all_columns]
)
body_positions[angle_columns] = np.degrees(body_positions[angle_columns])
body_positions

All angles are in degrees.

## Simulated visits

In [ ]:
visits = schedview.collect.multisim.read_multiple_prenights(visit_seq_archive_metadata, sim_date, day_obs, telescope=telescope)

# ColumnDataSource gets cannot accept a UUID, so convert it to a string
visits_ds = bokeh.models.ColumnDataSource(
#    visits.assign(visitseq_uuid=visits['visitseq_uuid'].apply(getattr, args=('hex', )))
    visits.assign(visitseq_uuid=visits['visitseq_uuid'].apply(str))
)

In [ ]:
sim_labels = visits['label'].unique()
sim_color_mapper, sim_color_dict, sim_marker_mapper, sim_hatch_dict = schedview.plot.generate_sim_indicators(sim_labels)

In [ ]:
with pd.option_context('display.max_colwidth', 1000):
    display(visits.groupby('sim_index').agg({'label': 'first', 'tags': 'first'}))

## Altitude and airmass

In [ ]:
fig = schedview.plot.multisim.plot_alt_airmass_vs_time(
    visits_ds,
    scatter_user_kwargs=dict(
        fill_alpha=0.2,
        color={"field": "label", "transform": sim_color_mapper},
        marker={"field": "label", "transform": sim_marker_mapper},
    )
)
bokeh.io.show(fig)

## Often repeated fields

An often repeated field is a field repeated at least four times in at least one simulation, where a "field" is a unique combination field coordinates and filter.

In [ ]:
often_repeated_fields, often_repeated_field_stats = schedview.compute.often_repeated_fields(visits)
often_repeated_field_stats.style.format({
    'first_time': lambda t: t.strftime("%H:%M:%S"),
    'last_time': lambda t: t.strftime("%H:%M:%S")}) 

At present, field coordinates must be exactly matched to be recognized as the "same" field.
A more robust approach would be to find clusters of nearby pointings (maybe with kmeans or a similar algorithm), and group by the identified clusters.

## Distribution comparisons

Overplotting distributions using kernel density estimates (similar to histograms, but continuous estimates of the underlying PDF).

KDEs are show here instead of histograms because they can be easier to intrepret for multiple overplotting distributions, if those distributions are actually different.

In [ ]:
try:
    fig = schedview.plot.overplot_kernel_density_estimates(visits, column='fieldRA', x_points=np.arange(0, 360), colors=sim_color_dict, hatches=sim_hatch_dict, bandwidth=1)
    bokeh.io.show(fig)
except Exception as e:
    print(e)

In [ ]:
try:
    fig = schedview.plot.overplot_kernel_density_estimates(visits, column='fieldDec', x_points=np.arange(-90, 30), colors=sim_color_dict, hatches=sim_hatch_dict, bandwidth=1)
    bokeh.io.show(fig)
except Exception as e:
    print(e)

In [ ]:
try:
    fig = schedview.plot.overplot_kernel_density_estimates(visits, column='airmass', bandwidth=0.001, x_points=np.arange(1.0, 2.5, 0.005), colors=sim_color_dict, hatches=sim_hatch_dict)
    bokeh.io.show(fig)
except Exception as e:
    print(e)

## Common visits

`sim_index` columns in the tables that follow refer to simulations with the following labels:

In [ ]:
with pd.option_context('display.max_colwidth', 512):
    display(visits.groupby('sim_index')['label'].first().to_frame())

In [ ]:
visit_counts = schedview.compute.multisim.count_visits_by_sim(visits)

Coordinate/filter/exposure time combinations repeated more that four times in any simulation:

In [ ]:
visit_counts.loc[visit_counts.max(axis='columns')>4, :]

Coordinate/filter/exposure time combinations all simulations have in common, statics on how often they occur:

In [ ]:
if len(visits):
    (visit_counts
    .T.describe().T
    .rename(columns={'min': 'min_visits'})
    .query('min_visits>0')
    .rename(columns={'min_visits': 'min'})
    .loc[:, ['min', '25%', '50%', 'mean', '75%', 'max']]
    .sort_values('min', ascending=False)
    ) 
else:
    print("No visits.")

Matrix of fraction of coordinate/filter/exposure time combinations present in one simulation that are alse present in another.

For example, column 1, row 2 has the fraction of such combinations present in simulation 1 that are also present is simulation 2.

In [ ]:
if len(visits):
    schedview.compute.multisim.fraction_common(visit_counts, visit_counts.columns[0], visit_counts.columns[1], match_count=False).item()
else:
    print("No visits.")

In [ ]:
schedview.compute.multisim.make_fraction_common_matrix(visit_counts, match_count=False)

Matrix of fraction of coordinate/filter/exposure time combinations present in one simulation that are alse present in another, where repeats in both are considered additional matches and differences in number of repeats of a given combination are counted as occurrences in one but not the other.

In [ ]:
schedview.compute.multisim.make_fraction_common_matrix(visit_counts)    

## Timing offsets

The following table shows the stistics for differences in timing (in seconds) in corresponding visits between a reference simulation and each other simulation.

In matching visits to find which ones in different simulations correspond to each other, each visit is counted only once.
When there are different total numbers of visits to the same field, the required number of visits of the simulation with more are dropped before matches are made.
When the total combinations to be checked is small, visits to be dropped are selected to be optimal to make the remainder of the visits match.
When there are too many total combinations to check in reasonable time, visits are dropped from the beginning or end.

In [ ]:
if len(visits):
    reference_sim_index = -1
    for sim_index, tags in visits.groupby('sim_index').agg({'tags': 'first'}).iterrows():
        if 'nominal' in tags and 'ideal' in tags and reference_sim_index < sim_index:
            reference_sim_index = visits.sim_index
    if reference_sim_index == -1:
        reference_sim_index = visits.sim_index.min()

    print(f"Reference sim index: {reference_sim_index}")
    matched_visit_dt_stats = schedview.compute.compute_matched_visit_delta_statistics(visits, reference_sim_index)
    matched_visit_dt_stats
else:
    print("No visits.")